# Automatización de Notificaciones de Traslados BDUA

## Descripción General
Este notebook automatiza el proceso completo de **notificación masiva** relacionado con traslados de salida (aprobados y negados) en procesos BDUA. Abarca desde la generación de documentos hasta el **envío automatizado de correos electrónicos masivos**, gestionando comunicaciones multicanal desde la EPS de origen hacia los usuarios finales.

## Objetivos
- **Enviar correos electrónicos masivos** de forma automatizada con notificaciones personalizadas por usuario
- Extraer y gestionar **números de teléfono** para notificaciones vía SMS
- Crear **reportes en PDF** individualizados adjuntos a cada notificación
- Consolidar información en **archivo Excel** para seguimiento y trazabilidad

## Insumos de Entrada
| Insumo | Descripción |
|--------|-------------|
| Base de traslados BDUA | Registros de traslados aprobados y negados |
| Información de usuarios | Correos electrónicos, números de teléfono |
| Datos EPS de origen | Información complementaria de las EPS |

## Productos de Salida
1. **Correos masivos enviados**: Notificaciones automáticas con PDF adjunto por cada usuario
2. **PDF por usuario**: Documentos de notificación personalizados con detalle del traslado
3. **Excel consolidado**: Tabla de seguimiento con:
   - Información del usuario
   - Números de teléfono y correos electrónicos
   - Estado del traslado
   - Registro de notificaciones enviadas

## Estructura del Notebook
1. Carga y validación de datos fuente
2. Procesamiento de información de usuarios
3. Generación de PDF de notificaciones
4. Extracción de contactos (email y teléfono)
5. **Envío automatizado de correos masivos** con PDF adjunto
6. Consolidación de resultados en archivo Excel
7. Reportes de ejecución y trazabilidad

# Mudolos

In [ ]:
# Módulos necesarios
import os  # Trabajar con rutas del sistema
import pandas as pd  # Trabajar con DataFrames
import datetime  # Manejo de fechas
from pathlib import Path  # Manejo de rutas
import smtplib  # Envío de correos
from email.mime.text import MIMEText  # Crear correos con texto
from email.mime.multipart import MIMEMultipart  # Correos con múltiples partes
from PyPDF2 import PdfReader  # Leer archivos PDF
import re  # Para validación de correos electrónicos

# Rutas y Variables

In [ ]:
# Variables
Periodo_notificación = "01/01/2026"
Fecha_Correo = "01/01/2026"
Informe = "Informe #03"

# Base de datos Procesos BDUA traslados de entrada aprobados y negados
R_s1_automatico = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Procesos BDUA EPS\Automatico-S1\All-AUTO-S1.txt"
R_s1_val = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Procesos BDUA EPS\S1.val\All-S1-VAL.txt"
R_s5 = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Procesos BDUA EPS\S5\S5 TXT\S5_consolidado.txt"

# Maestro SIE Correos y Telefonos
R_maestro_sie = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\SIE\Aseguramiento\ms_sie\Reporte_Validación Archivos Maestro_2026_02_21.csv"

# Ruta Salida
R_salida = fr"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\informes\2026\CTO 102.2026\CTO102.2026 {Informe}\12 Actividad\Bases de datos notificaciones telefonicas\TRASLADOS APROBADOS Y NEGADOS BDUA"


# Cargue de dataframes

In [ ]:
df_s1_automatico = pd.read_csv(R_s1_automatico, sep=",", encoding="latin-1", dtype=str)
df_s1_val = pd.read_csv(R_s1_val, sep=",", encoding="latin-1", dtype=str)
df_s5 = pd.read_csv(R_s5, sep=",", encoding="latin-1", dtype=str)

df_ms_sie = pd.read_csv(R_maestro_sie, sep=';', encoding='ANSI', header=0, dtype=str)

# Limpieza de datos

## Procesos BDUA

### Fecha a reportar

In [ ]:
# Extraer mes y año del periodo de notificación
mes_filtro = int(Periodo_notificación.split("/")[1])  # Mes
anio_filtro = int(Periodo_notificación.split("/")[2])  # Año

print(f"{'='*60}")
print(f"FILTRO APLICADO: Mes={mes_filtro:02d} | Año={anio_filtro}")
print(f"{'='*60}\n")

# Función para filtrar por mes y año y mostrar resumen
def filtrar_por_periodo(df, nombre_df, col_fecha="FECHA_PROCESO"):
    """Filtra el DataFrame por mes/año y muestra resumen en consola."""
    registros_antes = len(df)
    
    # Convertir la columna de fecha a datetime para extraer mes y año
    fecha_parsed = pd.to_datetime(df[col_fecha], format="%d/%m/%Y", dayfirst=True)
    
    # Filtrar por mes y año
    mask = (fecha_parsed.dt.month == mes_filtro) & (fecha_parsed.dt.year == anio_filtro)
    df_filtrado = df[mask].copy()
    
    registros_despues = len(df_filtrado)
    
    # Reporte en consola
    print(f"📋 {nombre_df}")
    print(f"   Registros antes: {registros_antes:,} → Después: {registros_despues:,}")
    print(f"   Distribución por FECHA_PROCESO:")
    
    conteo_fechas = df_filtrado[col_fecha].value_counts().sort_index()
    for fecha, cantidad in conteo_fechas.items():
        print(f"     • {fecha}: {cantidad:,} registros")
    
    print(f"   Total fechas únicas: {len(conteo_fechas)}")
    print(f"{'-'*60}")
    
    return df_filtrado

# Aplicar filtro a los 3 DataFrames
df_s1_automatico = filtrar_por_periodo(df_s1_automatico, "df_s1_automatico")
df_s1_val = filtrar_por_periodo(df_s1_val, "df_s1_val")
df_s5 = filtrar_por_periodo(df_s5, "df_s5")

# Resumen final
print(f"\n{'='*60}")
print(f"RESUMEN FINAL DEL FILTRO")
print(f"{'='*60}")
print(f"  df_s1_automatico : {len(df_s1_automatico):>8,} registros")
print(f"  df_s1_val        : {len(df_s1_val):>8,} registros")
print(f"  df_s5            : {len(df_s5):>8,} registros")
print(f"  {'─'*40}")
print(f"  TOTAL            : {len(df_s1_automatico) + len(df_s1_val) + len(df_s5):>8,} registros")

### Filtrar Traslados

In [ ]:
# Filtrar traslados de EPS (eliminar movilidades)
print(f"{'='*60}")
print(f"FILTRO: Traslados de EPS (eliminar movilidades)")
print(f"  Excluir TIPO_TRASLADO: 3, 4, 5")
print(f"  Excluir ENT_ID_ORIGEN: EPS025, EPSC25")
print(f"{'='*60}\n")

tipos_excluir = ["3", "4", "5"]
entidades_excluir = ["EPS025", "EPSC25"]

def filtrar_traslados_eps(df, nombre_df):
    """Filtra movilidades y entidades propias, muestra resumen."""
    registros_antes = len(df)
    
    # Aplicar filtros: excluir tipos de traslado y entidades
    mask = (
        ~df["TIPO_TRASLADO"].isin(tipos_excluir) &
        ~df["ENT_ID_ORIGEN"].isin(entidades_excluir)
    )
    df_filtrado = df[mask].copy()
    
    registros_despues = len(df_filtrado)
    eliminados = registros_antes - registros_despues
    
    # Reporte en consola
    print(f"📋 {nombre_df}")
    print(f"   Registros antes: {registros_antes:,} → Después: {registros_despues:,} (eliminados: {eliminados:,})")
    
    print(f"\n   Distribución por TIPO_TRASLADO:")
    conteo_tipo = df_filtrado["TIPO_TRASLADO"].value_counts().sort_index()
    for tipo, cantidad in conteo_tipo.items():
        print(f"     • Tipo {tipo}: {cantidad:,} registros")
    
    print(f"\n   Distribución por ENT_ID_ORIGEN:")
    conteo_entidad = df_filtrado["ENT_ID_ORIGEN"].value_counts().sort_index()
    for entidad, cantidad in conteo_entidad.items():
        print(f"     • {entidad}: {cantidad:,} registros")
    
    print(f"{'-'*60}")
    
    return df_filtrado

# Aplicar filtro a los 2 DataFrames
df_s1_automatico = filtrar_traslados_eps(df_s1_automatico, "df_s1_automatico")
df_s1_val = filtrar_traslados_eps(df_s1_val, "df_s1_val")

# Resumen final
print(f"\n{'='*60}")
print(f"RESUMEN DESPUÉS DE FILTRAR TRASLADOS EPS")
print(f"{'='*60}")
print(f"  df_s1_automatico : {len(df_s1_automatico):>8,} registros")
print(f"  df_s1_val        : {len(df_s1_val):>8,} registros")
print(f"  df_s5            : {len(df_s5):>8,} registros (sin cambios)")
print(f"  {'─'*40}")
print(f"  TOTAL            : {len(df_s1_automatico) + len(df_s1_val) + len(df_s5):>8,} registros")

### Unificar información en un solo datafarme S1.val

In [ ]:
# ============================================================
# CONSOLIDACIÓN DE TRASLADOS DE ENTRADA (Aprobados y Negados)
# ============================================================

print(f"{'='*70}")
print(f"CONSOLIDACIÓN DE TRASLADOS DE ENTRADA")
print(f"  S1-Automático: Aprobados automáticos por ADRES")
print(f"  S5: Respuesta EPS origen (0=Negado, 1=Aprobado)")
print(f"  S1-Val: Base maestra con información completa")
print(f"{'='*70}\n")

# Columnas clave para el cruce
cols_clave = ["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION", "FECHA_PROCESO"]

# ── PASO 1: Marcar aprobados automáticos desde S1-Automático ──
print(f"{'─'*70}")
print(f"PASO 1: Identificar aprobados automáticos (S1-Automático)")
print(f"{'─'*70}")

# Crear set de claves del S1-Automático para búsqueda eficiente
claves_s1_auto = set(
    df_s1_automatico[cols_clave].apply(lambda x: tuple(x), axis=1)
)
print(f"  Registros únicos en S1-Automático: {len(claves_s1_auto):,}")

# ── PASO 2: Preparar S5 (respuesta EPS origen) ──
print(f"\n{'─'*70}")
print(f"PASO 2: Preparar respuestas del S5")
print(f"{'─'*70}")

# Crear DataFrame de respuestas S5 con las columnas clave + RESPUESTA
df_s5_respuesta = df_s5[cols_clave + ["RESPUESTA"]].copy()
df_s5_respuesta["RESPUESTA"] = df_s5_respuesta["RESPUESTA"].str.strip()

conteo_s5 = df_s5_respuesta["RESPUESTA"].value_counts()
print(f"  Total registros S5: {len(df_s5_respuesta):,}")
for resp, cant in conteo_s5.items():
    etiqueta = "Aprobado" if resp == "1" else "Negado" if resp == "0" else f"Desconocido ({resp})"
    print(f"    • {etiqueta} (RESPUESTA={resp}): {cant:,}")

# Crear set de claves S5 para búsqueda
claves_s5 = set(
    df_s5_respuesta[cols_clave].apply(lambda x: tuple(x), axis=1)
)
print(f"  Registros únicos S5 por clave: {len(claves_s5):,}")

# ── PASO 3: Cruzar S1-Val con S1-Automático y S5 ──
print(f"\n{'─'*70}")
print(f"PASO 3: Cruzar S1-Val con S1-Automático y S5")
print(f"{'─'*70}")

df_consolidado = df_s1_val.copy()
registros_s1_val = len(df_consolidado)
print(f"  Registros en S1-Val: {registros_s1_val:,}")

# Crear tupla de claves en S1-Val
df_consolidado["_clave"] = list(
    df_consolidado[cols_clave].apply(lambda x: tuple(x), axis=1)
)

# Marcar origen: S1-Automático o S5
df_consolidado["ORIGEN_RESPUESTA"] = df_consolidado["_clave"].apply(
    lambda x: "S1-AUTOMATICO" if x in claves_s1_auto 
              else ("S5" if x in claves_s5 else "SIN_RESPUESTA")
)

# Cruzar con S5 para obtener la respuesta (merge por claves)
df_consolidado = df_consolidado.merge(
    df_s5_respuesta[cols_clave + ["RESPUESTA"]],
    on=cols_clave,
    how="left"
)

# Asignar estado del traslado
# S1-Automático → siempre Aprobado
# S5 → según columna RESPUESTA (1=Aprobado, 0=Negado)
# Sin respuesta → marcar como pendiente
df_consolidado["ESTADO_TRASLADO"] = df_consolidado.apply(
    lambda row: "APROBADO" if row["ORIGEN_RESPUESTA"] == "S1-AUTOMATICO"
                else ("APROBADO" if row["RESPUESTA"] == "1" 
                      else ("NEGADO" if row["RESPUESTA"] == "0" 
                            else "SIN_RESPUESTA")),
    axis=1
)

# Eliminar columna auxiliar
df_consolidado.drop(columns=["_clave"], inplace=True)

# Reporte de clasificación
print(f"\n  Clasificación por ORIGEN_RESPUESTA:")
conteo_origen = df_consolidado["ORIGEN_RESPUESTA"].value_counts()
for origen, cant in conteo_origen.items():
    print(f"    • {origen}: {cant:,}")

print(f"\n  Clasificación por ESTADO_TRASLADO:")
conteo_estado = df_consolidado["ESTADO_TRASLADO"].value_counts()
for estado, cant in conteo_estado.items():
    print(f"    • {estado}: {cant:,}")

# ── PASO 4: Deduplicación priorizando aprobados ──
print(f"\n{'─'*70}")
print(f"PASO 4: Deduplicación (priorizar aprobados sobre negados)")
print(f"{'─'*70}")

registros_antes_dedup = len(df_consolidado)

# Columnas de identificación del usuario (sin FECHA_PROCESO)
cols_usuario = ["TPS_IDN_ID", "HST_IDN_NUMERO_IDENTIFICACION"]

# Crear prioridad: APROBADO > NEGADO > SIN_RESPUESTA
prioridad_estado = {"APROBADO": 0, "NEGADO": 1, "SIN_RESPUESTA": 2}
df_consolidado["_prioridad"] = df_consolidado["ESTADO_TRASLADO"].map(prioridad_estado)

# Ordenar: por usuario, prioridad (aprobado primero), fecha proceso (más reciente primero)
df_consolidado["_fecha_orden"] = pd.to_datetime(
    df_consolidado["FECHA_PROCESO"], format="%d/%m/%Y", dayfirst=True
)
df_consolidado.sort_values(
    by=cols_usuario + ["_prioridad", "_fecha_orden"],
    ascending=[True, True, True, False],  # Prioridad ascendente (0=aprobado primero), fecha descendente
    inplace=True
)

# Mantener el primer registro por usuario (el de mayor prioridad y fecha más reciente)
df_consolidado.drop_duplicates(subset=cols_usuario, keep="first", inplace=True)

# Eliminar columnas auxiliares
df_consolidado.drop(columns=["_prioridad", "_fecha_orden"], inplace=True)

registros_despues_dedup = len(df_consolidado)
duplicados_eliminados = registros_antes_dedup - registros_despues_dedup

print(f"  Registros antes: {registros_antes_dedup:,}")
print(f"  Registros después: {registros_despues_dedup:,}")
print(f"  Duplicados eliminados: {duplicados_eliminados:,}")

# ── PASO 5: Validación de coherencia temporal ──
print(f"\n{'─'*70}")
print(f"PASO 5: Validación de coherencia temporal")
print(f"{'─'*70}")

print(f"\n  Distribución por FECHA_PROCESO:")
conteo_fechas = df_consolidado["FECHA_PROCESO"].value_counts().sort_index()
for fecha, cant in conteo_fechas.items():
    print(f"    • {fecha}: {cant:,} registros")

print(f"\n  Cruce ESTADO_TRASLADO × ORIGEN_RESPUESTA:")
tabla_cruzada = pd.crosstab(
    df_consolidado["ESTADO_TRASLADO"], 
    df_consolidado["ORIGEN_RESPUESTA"],
    margins=True,
    margins_name="TOTAL"
)
print(tabla_cruzada.to_string(col_space=15))

print(f"\n  Distribución ESTADO × FECHA_PROCESO:")
tabla_fecha_estado = pd.crosstab(
    df_consolidado["FECHA_PROCESO"], 
    df_consolidado["ESTADO_TRASLADO"],
    margins=True,
    margins_name="TOTAL"
)
print(tabla_fecha_estado.to_string(col_space=12))

# ── RESUMEN FINAL ──
print(f"\n{'='*70}")
print(f"RESUMEN FINAL - CONSOLIDADO DE TRASLADOS DE ENTRADA")
print(f"{'='*70}")
print(f"  Total registros consolidados : {len(df_consolidado):>8,}")
print(f"  ├── APROBADOS                : {len(df_consolidado[df_consolidado['ESTADO_TRASLADO'] == 'APROBADO']):>8,}")
print(f"  │   ├── S1-Automático        : {len(df_consolidado[(df_consolidado['ESTADO_TRASLADO'] == 'APROBADO') & (df_consolidado['ORIGEN_RESPUESTA'] == 'S1-AUTOMATICO')]):>8,}")
print(f"  │   └── S5                   : {len(df_consolidado[(df_consolidado['ESTADO_TRASLADO'] == 'APROBADO') & (df_consolidado['ORIGEN_RESPUESTA'] == 'S5')]):>8,}")
print(f"  ├── NEGADOS                  : {len(df_consolidado[df_consolidado['ESTADO_TRASLADO'] == 'NEGADO']):>8,}")
print(f"  └── SIN RESPUESTA            : {len(df_consolidado[df_consolidado['ESTADO_TRASLADO'] == 'SIN_RESPUESTA']):>8,}")
print(f"  {'─'*50}")
print(f"  Usuarios únicos              : {df_consolidado[cols_usuario].drop_duplicates().shape[0]:>8,}")
print(f"  Fechas de proceso            : {df_consolidado['FECHA_PROCESO'].nunique():>8,}")

### Eliminar dataframes, liberar espacio en RAM

In [ ]:
del df_s1_automatico, df_s1_val, df_s5  # Liberar memoria de DataFrames originales

## Depurar df_consolidado

### Eliminar Columnas

In [ ]:
# Eliminar columnas innecesarias del consolidado
columnas_a_eliminar = [
    "ENT_ID", "TPS_IDN_ID_2", "HST_IDN_NUMERO_IDENTIFICACION_2", "AFL_PRIMER_APELLIDO_2",
    "AFL_SEGUNDO_APELLIDO_2", "AFL_PRIMER_NOMBRE_2", "AFL_SEGUNDO_NOMBRE_2",
    "AFL_FECHA_NACIMIENTO_2", "TPS_GNR_ID_2", "TPS_MDL_SBS_ID"
]

df_consolidado.drop(columns=columnas_a_eliminar, inplace=True, errors="ignore")
print("Columnas innecesarias eliminadas del df_consolidado.")

## Correos y telefono

In [ ]:
import pandas as pd

def cruzar_maestro_sie(df_consolidado, df_ms_sie):
    """
    Realiza el cruce entre el consolidado y el maestro SIE.
    Aplica buenas prácticas de limpieza de llaves y auditoría de datos.
    """
    
    # 1. Estandarización de llaves (Prevención de fallos por espacios o tipos de datos)
    keys_consolidado = ['TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION']
    keys_sie = ['tipo_documento', 'numero_identificacion']
    
    for df, keys in [(df_consolidado, keys_consolidado), (df_ms_sie, keys_sie)]:
        for key in keys:
            df[key] = df[key].astype(str).str.strip()

    # 2. Selección de columnas necesarias del maestro para optimizar memoria
    cols_interes = keys_sie + ['celular', 'telefono_1', 'telefono_2', 'correo_electronico']
    df_ms_sie_subset = df_ms_sie[cols_interes].drop_duplicates(subset=keys_sie)

    # 3. Proceso de Cruce (Left Join)
    # Usamos left join para no perder registros del df_consolidado original
    df_resultado = pd.merge(
        df_consolidado,
        df_ms_sie_subset,
        left_on=keys_consolidado,
        right_on=keys_sie,
        how='left'
    )

    # 4. Auditoría de Calidad
    print("=== REPORTE DE CALIDAD DEL CRUCE ===")
    
    # Cantidad de registros que NO cruzaron (donde las columnas nuevas quedaron NaN)
    # Tomamos 'celular' como referencia, pero lo ideal es validar contra la llave del SIE
    no_cruzaron = df_resultado['tipo_documento'].isna().sum()
    total_registros = len(df_resultado)
    
    print(f"Total registros en consolidado: {total_registros}")
    print(f"Registros que NO se encontraron en SIE: {no_cruzaron} ({(no_cruzaron/total_registros)*100:.2f}%)")
    
    print("\n--- Conteo de datos recuperados (No nulos) ---")
    columnas_nuevas = ['celular', 'telefono_1', 'telefono_2', 'correo_electronico']
    for col in columnas_nuevas:
        conteo = df_resultado[col].notna().sum()
        print(f"Columna '{col}': {conteo} registros con información.")

    # 5. Limpieza post-cruce (opcional: eliminar llaves duplicadas del maestro)
    df_resultado = df_resultado.drop(columns=keys_sie)
    
    return df_resultado, no_cruzaron

# Ejemplo de uso:
df_consolidado, fallos = cruzar_maestro_sie(df_consolidado, df_ms_sie)

### Eliminar df_ms_sie

In [ ]:
del df_ms_sie

## Depurar telefonos

In [ ]:
import pandas as pd
import re

def limpiar_y_validar_telefonos(df, columnas):
    """
    Limpia y valida celulares colombianos en el DataFrame consolidado.
    Distingue entre datos que ya venían vacíos y datos que fueron eliminados por mala calidad.
    """
    reporte = {}

    def es_valido(numero):
        # Si quedó vacío tras la limpieza o era nulo, no es válido
        if not numero or numero == 'nan':
            return False
        # 1. Longitud 10, inicia con 3 y tiene al menos 4 dígitos distintos (Entropía)
        return (len(numero) == 10 and 
                numero.startswith('3') and 
                len(set(numero)) >= 4)

    for col in columnas:
        if col not in df.columns:
            continue
            
        # 1. Contamos cuántos datos REALES (no nulos) hay antes de empezar
        # Esto evita contar los NaN del merge fallido como "datos iniciales"
        datos_reales_iniciales = df[col].dropna().count()
        
        # 2. Limpieza de caracteres
        # Usamos fillna('') antes de convertir a str para evitar el texto "nan"
        df[col] = df[col].fillna('').astype(str).str.replace(r'\D', '', regex=True)
        
        # 3. Aplicamos validación de estructura y calidad
        mask_validos = df[col].apply(es_valido)
        
        # 4. Cálculo de métricas para Lumethik
        validados = mask_validos.sum()
        # Eliminados son los que tenían algo pero no pasaron la regla de calidad
        eliminados = datos_reales_iniciales - validados
        
        # 5. Limpieza final en el DataFrame: lo que no es válido se vuelve NaN real
        df.loc[~mask_validos, col] = None
        
        reporte[col] = {
            'iniciales': datos_reales_iniciales,
            'validados': validados,
            'eliminados': eliminados,
            'vacios_finales': df[col].isna().sum()
        }

    # --- Salida por Consola ---
    print("\n" + "="*80)
    print(f"{'AUDITORÍA DE CALIDAD TELEFÓNICA':^80}")
    print("="*80)
    print(f"{'COLUMNA':<20} | {'EXISTENTES':<12} | {'VALIDADOS':<12} | {'ELIMINADOS':<12} | {'NULOS FINAL'}")
    print("-" * 80)
    
    for col, stats in reporte.items():
        print(f"{col:<20} | {stats['iniciales']:<12} | {stats['validados']:<12} | {stats['eliminados']:<12} | {stats['vacios_finales']}")
    print("="*80 + "\n")
    
    return df

# Aplicación directa sobre tu consolidado
columnas_telefonos = ['celular', 'telefono_1', 'telefono_2']
df_consolidado = limpiar_y_validar_telefonos(df_consolidado, columnas_telefonos)

## Depurar Correos df_consolidado[correo_electronico]

In [ ]:
import pandas as pd
import re

def normalizar_y_validar_correos(df, columna='correo_electronico'):
    """
    Normaliza correos: minúsculas, corrige errores de puntuación y 
    dominios comunes mal escritos. Valida estructura final.
    """
    
    # Diccionario de correcciones comunes (Heurística de Lumethik)
    correcciones_dominios = {
        r'@gamil\.': '@gmail.',
        r'@gamail\.': '@gmail.',
        r'@gimal\.': '@gmail.',
        r'@gimail\.': '@gmail.',
        r'@hotmial\.': '@hotmail.',
        r'@hotmal\.': '@hotmail.',
        r'@outlok\.': '@outlook.',
        r'@outluk\.': '@outlook.',
        r'@msn\.con$': '@msn.com',
        r'\.con$': '.com',  # Error común de dedo
        r',com$': '.com',   # Error de coma por punto
    }

    def validar_estructura(email):
        if not email or email == 'nan':
            return None
        # Regex estándar para email (RFC 5322 simplificada)
        patron = r'^[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}$'
        if re.match(patron, email):
            return email
        return None

    # 1. Conteo inicial (datos que no son nulos)
    total_inicial = df[columna].dropna().count()

    # 2. Pre-procesamiento básico: Minúsculas y quitar espacios
    df[columna] = df[columna].fillna('').astype(str).str.lower().str.strip()

    # 3. Limpieza de errores de puntuación y dominios (Fuzzy Fix)
    # Reemplazamos comas por puntos antes de las correcciones de dominio
    df[columna] = df[columna].str.replace(',', '.', regex=False)
    
    for error, correccion in correcciones_dominios.items():
        df[columna] = df[columna].str.replace(error, correccion, regex=True)

    # 4. Validación de estructura final
    mask_validos = df[columna].apply(validar_estructura).notna()
    
    # Identificamos cuántos se "arreglaron" vs cuántos se eliminaron
    validados = mask_validos.sum()
    eliminados = total_inicial - validados

    # 5. Aplicar limpieza final (lo no válido a None)
    df.loc[~mask_validos, columna] = None

    # --- Reporte de Auditoría ---
    print("\n" + "="*60)
    print(f"{'REPORTE DE CALIDAD: CORREOS ELECTRÓNICOS':^60}")
    print("="*60)
    print(f"Registros con correo inicialmente:   {total_inicial}")
    print(f"Correos válidos (y corregidos):      {validados}")
    print(f"Correos eliminados (irreparables):   {eliminados}")
    print(f"Efectividad de recuperación:         {(validados/total_inicial)*100:.2f}%" if total_inicial > 0 else "N/A")
    print("-" * 60)
    
    return df

# Ejecución
df_consolidado = normalizar_y_validar_correos(df_consolidado)